In [15]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

db = SQLAlchemy(app)

In [16]:
engine = db.create_engine('sqlite:///datacamp.sqlite1', echo = True)
conn = engine.connect()
metadata = db.MetaData()

Student = db.Table('student', metadata,
                    db.Column('Id', db.Integer(), primary_key = True),
                    db.Column('Name', db.String(255), nullable = False),
                    db.Column('Major', db.String(255), default = 'Maths'),
                    db.Column('Pass', db.Boolean(), default = True)
                )
metadata.create_all(engine)

2024-02-28 11:50:31,856 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-28 11:50:31,857 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("student")
2024-02-28 11:50:31,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-28 11:50:31,859 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
output = conn.execute(Student.select()).fetchall()
print(output)

2024-02-28 11:50:31,866 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-28 11:50:31,867 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student
2024-02-28 11:50:31,867 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ()
[(11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True), (41, 'Rohit1', 'Science', False)]


In [18]:
query = db.insert(Student)
values = [{'Id':11, 'Name':'Ravi1', 'Major':'Science', 'Pass':True},
          {'Id':21, 'Name':'Raj1', 'Major':'Maths', 'Pass':False},
          {'Id':31, 'Name':'Rahul1', 'Major':'Arts', 'Pass':True},
          {'Id':41, 'Name':'Rohit1', 'Major':'Science', 'Pass':False}]

Result = conn.execute(query, values)

2024-02-28 11:50:31,877 INFO sqlalchemy.engine.Engine INSERT INTO student ("Id", "Name", "Major", "Pass") VALUES (?, ?, ?, ?)
2024-02-28 11:50:31,877 INFO sqlalchemy.engine.Engine [generated in 0.00075s] [(11, 'Ravi1', 'Science', 1), (21, 'Raj1', 'Maths', 0), (31, 'Rahul1', 'Arts', 1), (41, 'Rohit1', 'Science', 0)]


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: Student.Id
[SQL: INSERT INTO student ("Id", "Name", "Major", "Pass") VALUES (?, ?, ?, ?)]
[parameters: [(11, 'Ravi1', 'Science', 1), (21, 'Raj1', 'Maths', 0), (31, 'Rahul1', 'Arts', 1), (41, 'Rohit1', 'Science', 0)]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
output = conn.execute(Student.select()).fetchall()
print(output)

2024-02-28 11:17:47,719 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student
2024-02-28 11:17:47,720 INFO sqlalchemy.engine.Engine [cached since 0.02266s ago] ()
[(11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True), (41, 'Rohit1', 'Science', False)]


In [ ]:
Result.connection.commit()
Result.close()

2024-02-28 11:17:47,729 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
query = Student.select().where(Student.columns.Major == 'Maths')
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:17:47,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-28 11:17:47,743 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ?
2024-02-28 11:17:47,745 INFO sqlalchemy.engine.Engine [generated in 0.00284s] ('Maths',)
[(21, 'Raj1', 'Maths', False)]


In [ ]:
query = Student.select().where(db.and_(Student.columns.Major == 'Maths', Student.columns.Pass != True))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:17:47,755 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ? AND student."Pass" != 1
2024-02-28 11:17:47,756 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ('Maths',)
[(21, 'Raj1', 'Maths', False)]


In [ ]:
query = Student.select().where(db.or_(Student.columns.Major == 'Maths', Student.columns.Pass == True))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:17:47,765 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ? OR student."Pass" = 1
2024-02-28 11:17:47,766 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ('Maths',)
[(11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True)]


In [ ]:
query = Student.select().where(Student.columns.Major.in_(['Science', 'Maths']))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:17:47,775 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" IN (?, ?)
2024-02-28 11:17:47,777 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ('Science', 'Maths')
[(11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (41, 'Rohit1', 'Science', False)]


In [ ]:
query = Student.select().order_by(db.desc(Student.columns.Name))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:17:47,785 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student ORDER BY student."Name" DESC
2024-02-28 11:17:47,787 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ()
[(41, 'Rohit1', 'Science', False), (11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True)]


In [ ]:
print(Student.columns.keys())

['Id', 'Name', 'Major', 'Pass']


In [ ]:
print(repr(metadata.tables['student']))

Table('student', MetaData(), Column('Id', Integer(), table=<student>, primary_key=True, nullable=False), Column('Name', String(length=255), table=<student>, nullable=False), Column('Major', String(length=255), table=<student>, default=ScalarElementColumnDefault('Maths')), Column('Pass', Boolean(), table=<student>, default=ScalarElementColumnDefault(True)), schema=None)


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, func
from sqlalchemy.orm import sessionmaker
from  sqlalchemy.ext.declarative import declarative_base

#Create a new engine instance
engine = create_engine('sqlite:///example.db')

Base = declarative_base()

#Define a new table with a name, metadata, and several columns
class Sales(Base):
    _tablename_ = 'sales'
    id = Column(Integer, primary_key = True)
    product = Column(String)
    quantity  = Column(Integer)

#Create the table
Base.metadata.create_all(engine)

#Prepare data
data = [
    Sales(product = 'Apples', quantity = 5),
    Sales(product = 'Oranges', quantity = 7),
    Sales(product = 'Apples', quantity = 3),
    Sales(product = 'Bananas', quantity = 8),
    Sales(product = 'Apples', quantity = 6),
    Sales(product = 'Oranges', quantity = 9),
]

#Create a session
Session = sessionmaker(bind = engine)
session  = Session()

try:
    #Add data to the session
    session.add_all(data)

    #commit the changes
    session.commit()

    #create a select statement
    stmt = session.query(Sales.product, func.sum(Sales.quantity).label("total")).group_by(Sales.product)
    #execute the statement
    results = session.execute(stmt).fetchall()
    for row in results:
        print (row)
finally:
    #close the session
    session.close()

C:\Users\shail sharma\AppData\Local\Temp\ipykernel_5984\1843722782.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


InvalidRequestError: Class <class '__main__.Sales'> does not have a __table__ or __tablename__ specified and does not inherit from an existing table-mapped class.